In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.python.client import device_lib
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']
print(get_available_gpus())

tf.random.set_seed(0)

['/device:GPU:0']


In [3]:
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

train_data, test_data = imdb["train"], imdb["test"]

In [4]:
training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

for s, l in train_data:
    training_sentences.append(s.numpy().decode('utf8'))
    training_labels.append(l.numpy())
    
for s, l in test_data:
    testing_sentences.append(s.numpy().decode('utf8'))
    testing_labels.append(l.numpy())

In [5]:
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [6]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type = "post"
oov_tok = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index

sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length)

In [7]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [8]:
num_epochs = 5
model.fit(padded,
          training_labels_final,
          epochs=num_epochs,
          validation_data=(testing_padded, testing_labels_final))

Epoch 1/5
782/782 [==============================] - 3s 4ms/step - loss: 0.5073 - accuracy: 0.7239 - val_loss: 0.3431 - val_accuracy: 0.8500
Epoch 2/5
782/782 [==============================] - 3s 4ms/step - loss: 0.2433 - accuracy: 0.9057 - val_loss: 0.3620 - val_accuracy: 0.8430
Epoch 3/5
782/782 [==============================] - 3s 4ms/step - loss: 0.0948 - accuracy: 0.9749 - val_loss: 0.4603 - val_accuracy: 0.8257
Epoch 4/5
782/782 [==============================] - 3s 4ms/step - loss: 0.0234 - accuracy: 0.9969 - val_loss: 0.5390 - val_accuracy: 0.8241
Epoch 5/5
782/782 [==============================] - 3s 4ms/step - loss: 0.0060 - accuracy: 0.9995 - val_loss: 0.5964 - val_accuracy: 0.8278


In [9]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
bidirectional (Bidirectional (None, 64)                12544     
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 390       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 7         
Total params: 172,941
Trainable params: 172,941
Non-trainable params: 0
_________________________________________________________________


In [10]:
num_epochs = 30
model.fit(padded,
          training_labels_final,
          epochs=num_epochs,
          validation_data=(testing_padded, testing_labels_final))

Epoch 1/30
782/782 [==============================] - 10s 13ms/step - loss: 0.6663 - accuracy: 0.5616 - val_loss: 0.6047 - val_accuracy: 0.5768
Epoch 2/30
782/782 [==============================] - 10s 12ms/step - loss: 0.4846 - accuracy: 0.7681 - val_loss: 0.3728 - val_accuracy: 0.8410
Epoch 3/30
782/782 [==============================] - 10s 12ms/step - loss: 0.3108 - accuracy: 0.8750 - val_loss: 0.3523 - val_accuracy: 0.8448
Epoch 4/30
782/782 [==============================] - 10s 12ms/step - loss: 0.2415 - accuracy: 0.9091 - val_loss: 0.4234 - val_accuracy: 0.8356
Epoch 5/30
782/782 [==============================] - 10s 12ms/step - loss: 0.1964 - accuracy: 0.9295 - val_loss: 0.4459 - val_accuracy: 0.8276
Epoch 6/30
782/782 [==============================] - 10s 12ms/step - loss: 0.1595 - accuracy: 0.9446 - val_loss: 0.4742 - val_accuracy: 0.8290
Epoch 7/30
782/782 [==============================] - 10s 12ms/step - loss: 0.1385 - accuracy: 0.9533 - val_loss: 0.4938 - val_accuracy:

In [7]:
embedding_dim = 64
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 64)           640000    
_________________________________________________________________
bidirectional (Bidirectional (None, 120, 128)          66048     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 64)                41216     
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 751,489
Trainable params: 751,489
Non-trainable params: 0
_________________________________________________________________


In [8]:
num_epochs = 30
model.fit(padded,
          training_labels_final,
          epochs=num_epochs,
          validation_data=(testing_padded, testing_labels_final))

Epoch 1/30
782/782 [==============================] - 23s 30ms/step - loss: 0.4444 - accuracy: 0.7892 - val_loss: 0.3955 - val_accuracy: 0.8304
Epoch 2/30
782/782 [==============================] - 24s 31ms/step - loss: 0.2784 - accuracy: 0.8886 - val_loss: 0.3789 - val_accuracy: 0.8473
Epoch 3/30
782/782 [==============================] - 25s 32ms/step - loss: 0.1896 - accuracy: 0.9286 - val_loss: 0.4144 - val_accuracy: 0.8269
Epoch 4/30
782/782 [==============================] - 24s 31ms/step - loss: 0.1142 - accuracy: 0.9575 - val_loss: 0.6712 - val_accuracy: 0.8136
Epoch 5/30
782/782 [==============================] - 25s 32ms/step - loss: 0.0709 - accuracy: 0.9761 - val_loss: 0.6247 - val_accuracy: 0.8133
Epoch 6/30
782/782 [==============================] - 26s 33ms/step - loss: 0.0539 - accuracy: 0.9807 - val_loss: 0.7486 - val_accuracy: 0.8276
Epoch 7/30
782/782 [==============================] - 26s 34ms/step - loss: 0.0316 - accuracy: 0.9896 - val_loss: 0.8338 - val_accuracy: